In [17]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset
from dataset import ThyroidNodules
from torch.utils.data import DataLoader
from glob import glob
from trainer import Trainer
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    jaccard_score,
    precision_score,
    recall_score,
)
from eval import compute_loss,DiceLoss
from timeit import default_timer as timer
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils import weight_norm
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Batch size and image dimensions
BATCH_SIZE = 16
HEIGHT = 128
WIDTH = 128

# Hyperparameters
H = HEIGHT
W = WIDTH
DEFAULT_LR = 0.001

train_x= sorted(glob(os.path.join("/Users/eloise-em/Documents/Haris Ghafoor Archive/Research and Development/RnD/Thyroid Dataset/tn3k/trainval-image",'*')))
train_y= sorted(glob(os.path.join("/Users/eloise-em/Documents/Haris Ghafoor Archive/Research and Development/RnD/Thyroid Dataset/tn3k/trainval-mask",'*')))

valid_x= sorted(glob(os.path.join("/Users/eloise-em/Documents/Haris Ghafoor Archive/Research and Development/RnD/Thyroid Dataset/tn3k/test-image",'*')))
valid_y= sorted(glob(os.path.join("/Users/eloise-em/Documents/Haris Ghafoor Archive/Research and Development/RnD/Thyroid Dataset/tn3k/test-mask",'*')))

# valid_x = "/Users/eloise-em/Documents/Haris Ghafoor Archive/Research and Development/RnD/Thyroid Dataset/tn3k/test-image"
# valid_y = "/Users/eloise-em/Documents/Haris Ghafoor Archive/Research and Development/RnD/Thyroid Dataset/tn3k/test-mask"
# Create datasets and dataloaders for this fold
train_dataset = ThyroidNodules(
    train_x,
    train_y,
    image_size=(H, W),
)

valid_dataset = ThyroidNodules(
    valid_x,
    valid_y,
    image_size=(H, W),
)

train_loader = DataLoader(
    dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2
)
valid_loader = DataLoader(
    dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2
)
# device = torch.device(args.device)
# print("Device:", device)

# model = AttentionUNetppGradual()
# model = model.to(device)

# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, "min", patience=10, verbose=True
# )

In [14]:
trainer = Trainer(42)

In [15]:
def calculate_metrics(y_true, y_pred, threshold):
    # Ground truth
    y_true = y_true.cpu().numpy()
    y_true = y_true > threshold
    y_true = y_true.astype(np.uint8)
    y_true = y_true.reshape(-1)

    # Prediction
    y_pred = torch.sigmoid(y_pred)
    y_pred = y_pred.cpu().numpy()
    y_pred = y_pred > threshold
    y_pred = y_pred.astype(np.uint8)
    y_pred = y_pred.reshape(-1)

    score_jaccard = jaccard_score(y_true, y_pred)
    score_f1 = f1_score(y_true, y_pred)
    score_recall = recall_score(y_true, y_pred)
    score_precision = precision_score(y_true, y_pred)
    score_acc = accuracy_score(y_true, y_pred)

    return [score_jaccard, score_f1, score_recall, score_precision, score_acc]

In [16]:
trainer = Trainer(seed=7)
input = next(iter(trainer.train_loader))[0].to(trainer.device)
output = trainer.model(input)
target = next(iter(trainer.train_loader))[1].to(trainer.device)
loss = compute_loss(output, target)
print("output", output.shape)
print("target", target.shape)
print("loss", loss)


output torch.Size([32, 1, 128, 128])
target torch.Size([32, 1, 128, 128])
loss tensor(1.6175, device='mps:0', grad_fn=<AddBackward0>)


In [25]:
rng = np.random.RandomState(26)
seeds = rng.randint(10000, size=5)
seeds


array([4917, 5894, 9648, 9318, 7891])

In [18]:
trainer = Trainer(seed=7)


In [19]:
trainer.fit()

Epoch [10/11], Training Loss: 1.503756, Time (this epoch): 194.69 s


(Unet(
   (MaxPool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (Conv1): ConvBlock(
     (conv): Sequential(
       (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU(inplace=True)
       (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (5): ReLU(inplace=True)
     )
   )
   (Conv2): ConvBlock(
     (conv): Sequential(
       (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU(inplace=True)
       (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (5): ReLU(in